In [1]:
from pylogic import *
from pylogic.proposition.proof_search import proof_search

In [2]:
# Example usage
A = propositions("A", is_assumption=True)
B, C, D, E, F, G = propositions("B", "C", "D", "E", "F", "G")
p6 = Implies(And(Or(D, E), Or(A, F)), G, is_assumption=True)
A_imp_B = Implies(A, B, is_assumption=True)
B_imp_C = Implies(B, C, is_assumption=True)
C_imp_D = Implies(C, D, is_assumption=True)
kb = [
    # A,
    B.or_(C).assume(),
    # A_imp_B,
    # B_imp_C,
    B.implies(F).assume(),
    # C_imp_D,
    C.implies(G).assume(),
    # p6,
]
target = F.or_(C)
proof = proof_search(kb, target)
print(proof)


2 Trying to prove F \/ C from B \/ C
--------------------
B \/ C
B -> F
C -> G


1 Trying to prove F \/ C from B -> F
--------------------
B \/ C
B -> F
C -> G
B


1 Trying to prove B from C -> G
--------------------
B \/ C
B -> F
C -> G
B


C {Proposition(B), Or(Proposition(F), Proposition(C)), Proposition(C)} [Or(Proposition(B), Proposition(C)), Implies(Proposition(B), Proposition(F)), Implies(Proposition(C), Proposition(G)), Proposition(B)]
1 Trying to prove F \/ C from C -> G
--------------------
B \/ C
B -> F
C -> G
B
F


C {Proposition(C), Or(Proposition(F), Proposition(C))} [Or(Proposition(B), Proposition(C)), Implies(Proposition(B), Proposition(F)), Implies(Proposition(C), Proposition(G)), Proposition(B), Proposition(F)]
1 Trying to prove F from C -> G
--------------------
B \/ C
B -> F
C -> G
B
F


C {Proposition(F), Proposition(C)} [Or(Proposition(B), Proposition(C)), Implies(Proposition(B), Proposition(F)), Implies(Proposition(C), Proposition(G)), Proposition(B), Proposition

In [3]:
P, Q, R, S, T = propositions("P", "Q", "R", "S", "T")
p1 = P.implies(Q).and_(R.implies(S)).assume()
p2 = Q.or_(S).implies(T).assume()
p3 = P.or_(R).assume()
p4 = neg(T).assume()

target = neg(P).and_(neg(R))
kb = [
    p1,
    p2,
    p3,
    p4,
]
proof = proof_search(kb, target)
print(proof.deduced_from)

1 Trying to prove ~P /\ ~R from Q \/ S -> T
--------------------
(P -> Q) /\ (R -> S)
Q \/ S -> T
P \/ R
~T
P -> Q
R -> S


2 Trying to prove Q \/ S from P \/ R
--------------------
(P -> Q) /\ (R -> S)
Q \/ S -> T
P \/ R
~T
P -> Q
R -> S


1 Trying to prove Q \/ S from ~T
--------------------
(P -> Q) /\ (R -> S)
Q \/ S -> T
P \/ R
~T
P -> Q
R -> S
P


1 Trying to prove T from P -> Q
--------------------
(P -> Q) /\ (R -> S)
Q \/ S -> T
P \/ R
~T
P -> Q
R -> S
P


1 Trying to prove P from R -> S
--------------------
(P -> Q) /\ (R -> S)
Q \/ S -> T
P \/ R
~T
P -> Q
R -> S
P


1 Trying to prove T from R -> S
--------------------
(P -> Q) /\ (R -> S)
Q \/ S -> T
P \/ R
~T
P -> Q
R -> S
P
Q


1 Trying to prove ~Q from R -> S
--------------------
(P -> Q) /\ (R -> S)
Q \/ S -> T
P \/ R
~T
P -> Q
R -> S
P


1 Trying to prove contradiction from R -> S
--------------------
(P -> Q) /\ (R -> S)
Q \/ S -> T
P \/ R
~T
P -> Q
R -> S
P
Q


1 Trying to prove T from R -> S
--------------------
(P -

In [4]:
A, B = propositions("A", "B")
p1 = A.or_(B).assume()
p2 = neg(B).assume()

target = A
kb = [
    p1,
    p2,
]
proof = proof_search(kb, target)
print(proof.deduced_from)

2 Trying to prove A from A \/ B
--------------------
A \/ B
~B


1 Trying to prove A from ~B
--------------------
A \/ B
~B
A


3 Goal A proven by case 0 on A \/ B
1 Trying to prove A from ~B
--------------------
A \/ B
~B
B


3 Goal A proven by case 1 on A \/ B
5 Goal A proven by cases on A \/ B
('by_cases', Or(Proposition(A), Proposition(B)), 'has inner contexts')


In [11]:
A, B, C, D, E, F = propositions("A", "B", "C", "D", "E", "F")
settings["USE_CLASSICAL_LOGIC"] = True

# Premises
p1 = A.implies(B.and_(F)).assume()
p2 = D.implies(neg(B)).assume()
p3 = C.or_(D).assume()
p4 = neg(E).implies(neg(D)).assume()

# Target
target = A.implies(C)

kb = [
    p1,
    p2,
    p3,
    # p4,
]

proof = proof_search(kb, target)
print(proof.deduced_from.inner_contexts[0]._proven[-1].deduced_from.inner_contexts[0].get_first_proven())


1 Trying to prove A -> C from A -> B /\ F
--------------------
A -> B /\ F
D /\ B -> contradiction
C \/ D


1 Trying to prove A from D /\ B -> contradiction
--------------------
A -> B /\ F
D /\ B -> contradiction
C \/ D


2 Trying to prove D /\ B from C \/ D
--------------------
A -> B /\ F
D /\ B -> contradiction
C \/ D


4 Goal D /\ B NOT proven by case 0 on C \/ D
2 Trying to prove D from C \/ D
--------------------
A -> B /\ F
D /\ B -> contradiction
C \/ D


4 Goal D NOT proven by case 0 on C \/ D
2 Trying to prove ~contradiction from C \/ D
--------------------
A -> B /\ F
D /\ B -> contradiction
C \/ D


3 Goal ~contradiction proven by case 0 on C \/ D
3 Goal ~contradiction proven by case 1 on C \/ D
5 Goal ~contradiction proven by cases on C \/ D
2 Trying to prove A from C \/ D
--------------------
A -> B /\ F
D /\ B -> contradiction
C \/ D
~(D /\ B)


1 Trying to prove A from ~(D /\ B)
--------------------
A -> B /\ F
D /\ B -> contradiction
C \/ D
~(D /\ B)
C


4 Goal A NOT 